<a href="https://colab.research.google.com/github/SIAT-SyM-Group/2025-dxpBGC-mining/blob/main/workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Genome-mining workflow

This notebook implements a reproducible genome-mining pipeline tailored for **biosynthetic gene cluster (BGC)** discovery:
1. Provide one or more protein query sequences (FASTA or plain sequence).
2. Run NCBI BLASTP (URLAPI) to obtain protein hits.
3. Map protein hits to genomic context (IPG/nuccore), download flanking regions, and write local FASTA.
4. Run antiSMASH on extracted neighborhoods.
5. Build a local BGCs diamond database and run **cblaster** searches.
6. Apply a rule-based filter and collect high-confidence candidate regions for downstream analysis.

**How to run:** fill in the forms below, then go to `Runtime → Run all`.

> Notes
> - NCBI **email is required** for Entrez.
> - Set an **NCBI API key** for higher rate limits (recommended).
> - The pipeline is written to be Colab-friendly: no manual shell scripting is required.



In [ ]:
#@title 1) Install Python dependencies and configure NCBI credentials
#@markdown Provide NCBI credentials here (email is required; API key is optional but recommended for higher rate limits).
#@markdown
#@markdown Options:
#@markdown - Fill the form fields below, or
#@markdown - Set environment variables NCBI_EMAIL / NCBI_API_KEY before running.

from __future__ import annotations

import os
import sys
import subprocess
import pkgutil
from typing import Optional

def _pip_install(*pkgs: str) -> None:
    cmd = [sys.executable, "-m", "pip", "install", "-q", *pkgs]
    subprocess.check_call(cmd)

def ensure_import(pkg: str, import_name: Optional[str] = None, pip_name: Optional[str] = None) -> None:
    name = import_name or pkg
    if pkgutil.find_loader(name) is None:
        _pip_install(pip_name or pkg)

# Core dependencies used across cells
ensure_import("requests")
ensure_import("tqdm", "tqdm")
ensure_import("biopython", "Bio", "biopython")

# Optional convenience dependencies (uncomment if needed)
# ensure_import("pandas")
# ensure_import("numpy")

# -------------------- Credential configuration --------------------
NCBI_EMAIL: str = ""  #@param {type:"string"}
NCBI_API_KEY: str = ""  #@param {type:"string"}

# Fallback to CONTACT_EMAIL from cell 0, if available
if not NCBI_EMAIL:
    NCBI_EMAIL = globals().get("EMAIL", "").strip()

if not NCBI_EMAIL:
    raise ValueError("NCBI_EMAIL is required. Set it in this cell or via the NCBI_EMAIL environment variable.")

os.environ["NCBI_EMAIL"] = NCBI_EMAIL
if NCBI_API_KEY:
    os.environ["NCBI_API_KEY"] = NCBI_API_KEY

# Configure Biopython Entrez defaults (used in later cells)
from Bio import Entrez
Entrez.email = NCBI_EMAIL
if NCBI_API_KEY:
    Entrez.api_key = NCBI_API_KEY

print("[INFO] Python deps: OK")
print(f"[INFO] NCBI_EMAIL set: {NCBI_EMAIL}")
print(f"[INFO] NCBI_API_KEY set: {'YES' if bool(NCBI_API_KEY) else 'NO'}")


In [ ]:
#@title 2) Provide protein query sequence(s) and write a standardized FASTA
#@markdown Paste either a raw amino-acid sequence (no header) or a FASTA (starting with '>').
#@markdown - **jobname** is used as the FASTA record ID when a raw sequence is provided.
#@markdown - The output FASTA (**out_fasta**) is used by the subsequent BLASTP cell.

import os, re, time, threading, xml.etree.ElementTree as ET
from pathlib import Path
from dataclasses import dataclass
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed

from tqdm.auto import tqdm
from Bio import Entrez, SeqIO

query_sequence = "MSHTPSQPQPSTDKKADKRLNKRLKDMPIAIVGMASIFANSRYLNKFWDLICDKIDAITDVPASHWAIDDYYDVDKSKADKSYCKRGGFMPEVDFNPMEFGLPPNILELTDSSQLLSLVVAKEVLQDANLPDDYDRDRIGITLGIGGGQKLSHSLNARLQYPVLKKVFKSSGLSDEDSELLIKKFQDQYVHWEENSFPGSLGNVIAGRIANRFDLGGMNCVVDAACAGSLAAMRMALTELTEGRSDMMITGGVCTDNSPYMYMSFSKTPAFTTNEQIQPFDIDSKGMMIGEGIGMVALKRLDDAERDGDRIYAVIKGVGASSDGKFKSIYAPRPEGQAKALERAYDDAGFAPHTVGLIEAHGTGTAAGDVAEFTGLSSVFSQDNAQLQHIALGSVKSQVGHTKSTAGTAGVIKAALALHHKVLPPTINVSKPNPKLEIDRSPFYLNTEARPWIQRSDDTPRRAGISSFGFGGTNFHLVLEEYRPDHTRDDAYRQRSVAQILLFAANDKTLLLNELKAVLQQASSAKAELSEAHFIQFAKPYALREITPQSARLGFIAKDYAQLQTLLTQAIAQLEANNAESWQLPSGISYRAKALVNEQTKIAALFAGQGAQYLNMGLELANNFPELRRHIHASDKVFSTHGKPALSSVLYPIPAFDDESIKAQETALTNTLYAQSAIGALSMAQYALFTQAGFAPDMLAGHSFGELSALCAAGVISMDDYIKLAFERGQAMAQSSQDTDAGVMYAVILKQKQDIEVINGCLAQFEGVKVANYNSPTQLVIAGASAATQQAAKAISELGFKAIALPVSGAFHTPLVAHAQKPFSAAIDKAQFNTPKIALYANGTGQLHPIDANAIKAALKDHMLQSVHFSEQLEAMYAAGARVFVEFGPKNILQKLTENTLAAQLNELCIISINPNPKGDSDSQLRSAAVQLAVAGVKLREIDPYQAELIAPAATSAMNIKLNATNYISPATRSKMVDSLQSGKITSQVQYVDRIVEKVVEKVVEKPVIVEKILEKVVEVEKPVAQNSNNIQQQTPAQPASFTAGQTNQDALSAFFAAQTQAAQLHQQFLAIPQQYGDTVSALMAEQAKMASLGIAIPESLQRSMELFHQHQAQTLKSHSDFMQLQTSSSQAVLALLGQMPASQVQAPIQAAAPVAVAVTKPVVPAQAPVVQGLAAEPKVTAVPVSEPTVQQPQVALAQVAQTKVTQPPLAQPQVQTVAAQTSALQVKPALQQIEHAMLSVVADKTGYPVEMLELSMDMEADLGIDSIKRVEILGTVQDELPNLPELSPEDLAECRTLGEIVALFSQAAPVTSATTVSHATQSAVAASAAVSNDEIERTMMAVVADKTGYPVEMLELSMDMEADLGIDSIKRVEILGTVQDELPNLPELSPEDLAECRTLGEIVALFSQAVPVAAQTFAAMAATNPQVVASAVTPIAAVSDGEIEHTMMAVVADKTGYPVEMLELSMDMEADLGIDSIKRVEILGTVQDKLPNLPELSPEDLAECRTLGEIAALFSQAAPVTAAATVSHATQSAIAARAAVSNDEIERTMMAVVADKTGYPVEMLELSMDMEADLGIDSIKRVEILGTVQDQLPNLPELSPEDLAECRTLGEIVALYAGSQSSSEALQQNHAATIQETQEAIAKTVEETIDLPPHSEVMLKKLPAAAELARIIATSDVQLTANSYVVIGDDGHNAGVIAEKLHAQGVKVAVVRSPKTVVTSASPLDSHIASFTLEAIDDESICEVINQIEALGQIAGFIHLQPQHKSVADKGAGLVLVDEAKASVEQAFLFAKFLQPLLTERDYCRFVTVSCIDGGFGYIGMDESVGALISQSELNQAALFGLTKTLNHEWPGVVCRALDIAPNLDAKTVANAVVQEYYLQDAPVEVGIDSNFDRVTLVAGTAALRHPPAVLSNADKILVTGGAKGVTFECALSLAKRCQAHFFLAGRSAHQVIPAWAEGKKSNELKAAAIAHLQSLGDKPTPKQVDALVWPVQSSLEISHALAAFDAIGASAEYLSVDVNDPAAIASTIAPINALSPITGIIHGAGVLADKHIQDKTLNEFERVYGTKVTGLNNLLSTLDLSQVKLIALFSSAAGFYGNTGQSDYAMSNDILNKAALQLAQQLPQAKVMSFDWGPWDGGMVNPALKKMFIDRGVYVIPLKAGAELFASQLLSDTGAQLLVGTDMQGNTANAVEVASAKKPEADLATALDPQPMAQTVPQSIRVMRSLDPKRMSFIEDHCINGHAVLPTVCAIDWMREAAKAHLGTAVSVSDYRLLKGVIFDEALLARNAPIELELMLTPLADAAQQSTEALAALISFEGRPQYQAVLVAQTDDMPDAQRFEVGELHSLIQEMAQQPAIANRESLYSDGTLFHGPRLQGISEVLTFDDQHLMAKVELPQVALDDCGKFAPKLEDKGTQPFAEDLLLQAMLVWARLKYQAASLPSTIGEFVSYAPLSFGEKAVLVLDVLKHSSRSLEANIALYHQDGRLSCEMKRAKVTISKTLNQAFLANKPQQLAQVQASIQNMAEVSVK"  #@param {type:"string", multiline:true}
jobname = "AAN54658.1-PfaA"  #@param {type:"string"}
out_fasta = "AAN54658.1-PfaA.faa"  #@param {type:"string"}
wrap_width = 80  #@param {type:"integer"}

import re, textwrap
from pathlib import Path

def _clean_seq(s: str) -> str:
    s = (s or "").strip()
    s = re.sub(r"\s+", "", s)
    return s.upper()

def _sanitize_id(s: str) -> str:
    s = (s or "").strip()
    if not s:
        raise ValueError("jobname must not be empty (it is used as the FASTA Query ID).")
    s = re.sub(r"\s+", "_", s)
    s = s.replace(">", "_")
    return s

def write_standard_fasta(query_sequence: str, jobname: str, out_fasta: str, wrap_width: int = 60):
    out_path = Path(out_fasta)

    qs = (query_sequence or "").strip()
    if not qs:
        raise ValueError("query_sequence must not be empty.")

    if re.search(r"^\s*>", qs, flags=re.MULTILINE):
        lines = qs.splitlines()
        records = []
        cur_h = None
        cur_s = []
        for ln in lines:
            if ln.startswith(">"):
                if cur_h is not None:
                    seq = _clean_seq("".join(cur_s))
                    records.append((cur_h, seq))
                cur_h = ln.strip()
                cur_s = []
            else:
                cur_s.append(ln.strip())
        if cur_h is not None:
            seq = _clean_seq("".join(cur_s))
            records.append((cur_h, seq))

        if not records:
            raise ValueError("Detected '>' but failed to parse a valid FASTA record.")

        with out_path.open("w", encoding="utf-8") as f:
            for h, seq in records:
                f.write(h + "\n")
                f.write("\n".join(textwrap.wrap(seq, width=int(wrap_width))) + "\n")

        preview_header = records[0][0]
        preview_len = len(records[0][1])

    else:
        qid = _sanitize_id(jobname)
        seq = _clean_seq(qs)
        if not seq:
            raise ValueError("Sequence becomes empty after cleaning. Please confirm you pasted a valid sequence.")

        with out_path.open("w", encoding="utf-8") as f:
            f.write(f">{qid}\n")
            f.write("\n".join(textwrap.wrap(seq, width=int(wrap_width))) + "\n")

        preview_header = f">{qid}"
        preview_len = len(seq)

    print(f"[OK] Wrote standard FASTA: {out_path.resolve()}")
    print(f"[OK] First record: {preview_header}  (length={preview_len})")
    print("\n--- Preview (first 20 lines) ---")
    print("\n".join(out_path.read_text(encoding="utf-8").splitlines()[:20]))

    try:
        from google.colab import files
        files.download(str(out_path))
    except Exception:
        pass

write_standard_fasta(query_sequence, jobname, out_fasta, wrap_width)

In [ ]:
#@title 3) Run BLASTP via NCBI BLAST URLAPI (PROGRAM is fixed to blastp)
#@markdown This cell submits the query FASTA to NCBI BLAST URLAPI, polls for completion, and downloads tabular results.
#@markdown It automatically uses **out_fasta** from the previous cell when available.


import os, time, re
import requests
from IPython.display import clear_output

BLAST_URL = "https://blast.ncbi.nlm.nih.gov/Blast.cgi"

PROGRAM = "blastp"

DATABASE   = "refseq_protein"  #@param {type:"string"}
HIT_TOP_N  = 10              #@param {type:"integer"}
EXPECT     = "1e-5"            #@param {type:"string"}

POLL_SEC   = 60
MAX_WAIT_H = 6
CLEAN_OLD  = True              #@param {type:"boolean"}

TIMEOUT_PUT = 180              #@param {type:"integer"}
TIMEOUT_GET = 300              #@param {type:"integer"}
# ==============================

query_fa = globals().get("out_fasta", globals().get("query_fa", "query.faa"))
jobname  = globals().get("jobname", "job")
EMAIL    = os.environ.get("NCBI_EMAIL", globals().get("EMAIL", "")).strip()
TOOL     = globals().get("TOOL", "symlab_notebook")

N = int(HIT_TOP_N)
MAX_WAIT = int(MAX_WAIT_H) * 3600

outf = f"{PROGRAM}.top{N}.tsv"
logf = f"{PROGRAM}.urlapi.log"
rawf = f"{PROGRAM}.last_raw.txt"
ridf = f"{PROGRAM}.rid.txt"

def log(line: str):
    with open(logf, "a", encoding="utf-8") as f:
        f.write(line.rstrip("\n") + "\n")

def tail_fast(path, n=25, block=8192):
    if not os.path.exists(path):
        return "(log not created yet)"
    with open(path, "rb") as f:
        f.seek(0, os.SEEK_END)
        end = f.tell()
        data = b""
        pos = end
        while pos > 0 and data.count(b"\n") <= n:
            step = min(block, pos)
            pos -= step
            f.seek(pos)
            data = f.read(step) + data
    return "\n".join(data.decode("utf-8", errors="replace").splitlines()[-n:])

def status_screen(phase: str, extra: str = ""):
    size = os.path.getsize(outf) if os.path.exists(outf) else 0
    clear_output(wait=True)
    print(time.strftime("%F %T"))
    print(f"phase: {phase}")
    print(f"query_fa: {query_fa}")
    print(f"db/program: {DATABASE} / {PROGRAM}")
    print(f"N/evalue: {N} / {EXPECT}")
    print(f"out: {size/1024/1024:.2f} MB  ({outf})")
    if extra:
        print(extra)
    print("\n--- last log lines ---")
    print(tail_fast(logf, n=25))

def parse_rid_rtoe(text: str):
    m_rid  = re.search(r"\bRID\s*=\s*([A-Z0-9]+)\b", text)
    m_rtoe = re.search(r"\bRTOE\s*=\s*(\d+)\b", text)
    rid  = m_rid.group(1) if m_rid else None
    rtoe = int(m_rtoe.group(1)) if m_rtoe else None
    return rid, rtoe

def parse_status_hits(text: str):
    ms = re.search(r"Status=([A-Z]+)", text)
    mh = re.search(r"ThereAreHits=([a-z]+)", text)
    return (ms.group(1) if ms else "NA"), (mh.group(1) if mh else "NA")

def extract_pre(text: str) -> str:
    m = re.search(r"<PRE>(.+?)</PRE>", text, re.DOTALL | re.IGNORECASE)
    return m.group(1) if m else text

def tabular_lines(text: str):
    out = []
    for line in text.splitlines():
        if not line or line.startswith("#"):
            continue
        if line.count("\t") >= 11:
            out.append(line)
    return out

def looks_like_tabular(lines):
    for ln in lines[:50]:
        cols = ln.split("\t")
        if len(cols) < 12:
            continue
        try:
            float(cols[2])   # pident
            float(cols[10])  # evalue
            float(cols[11])  # bitscore
            return True
        except Exception:
            pass
    return False

if not os.path.exists(query_fa):
    raise FileNotFoundError(
        f"FASTA file not found: {query_fa}\n"
        f"Run the previous cell to generate the FASTA (or set out_fasta/query_fa to an existing FASTA file)."
    )

if CLEAN_OLD:
    for f in (outf, logf, rawf, ridf):
        if os.path.exists(f):
            os.remove(f)

with open(query_fa, "r", encoding="utf-8", errors="replace") as f:
    fasta = f.read().strip()
if not fasta.startswith(">"):
    raise ValueError(f"{query_fa} is not a valid FASTA (must start with '>').")

sess = requests.Session()
sess.headers.update({"User-Agent": TOOL})
t0 = time.time()

# 1) SUBMIT (CMD=Put)
phase = "SUBMIT (CMD=Put)"
status_screen(phase, f"jobname={jobname}\nEMAIL={EMAIL}\nTOOL={TOOL}")
log(f"[{time.strftime('%F %T')}] {phase}")

put_params = {
    "CMD": "Put",
    "PROGRAM": PROGRAM,
    "DATABASE": DATABASE,
    "EXPECT": EXPECT,
    "HITLIST_SIZE": str(N),
    "ALIGNMENTS": str(N),
    "DESCRIPTIONS": str(N),
    "EMAIL": EMAIL,
    "TOOL": TOOL,
}

r = sess.post(BLAST_URL, params=put_params, files={"QUERY": (None, fasta)}, timeout=TIMEOUT_PUT)
r.raise_for_status()

RID, RTOE = parse_rid_rtoe(r.text)
if not RID:
    with open(rawf, "w", encoding="utf-8", errors="replace") as f:
        f.write(r.text)
    log("PUT response saved -> " + rawf)
    raise RuntimeError("Failed to parse RID (PUT response was written to rawf).")

with open(ridf, "w", encoding="utf-8") as f:
    f.write(RID + "\n")

log(f"[{time.strftime('%F %T')}] RID={RID} RTOE={RTOE if RTOE is not None else 'NA'}")
status_screen("SUBMITTED", f"RID={RID}\nRID saved -> {ridf}\nRTOE={RTOE if RTOE is not None else 'NA'}")

if RTOE:
    for left in range(RTOE, 0, -1):
        if left % 3 == 0:
            status_screen("WAIT (RTOE)", f"RID={RID}  left={left}s")
        time.sleep(1)

# 2) POLL
deadline = t0 + MAX_WAIT
tries = 0

while True:
    tries += 1
    if time.time() > deadline:
        raise TimeoutError(f"Timed out after MAX_WAIT={MAX_WAIT}s waiting for READY; RID={RID}")

    phase = "POLL (SearchInfo)"
    params = {"CMD": "Get", "RID": RID, "FORMAT_OBJECT": "SearchInfo", "EMAIL": EMAIL, "TOOL": TOOL}
    res = sess.get(BLAST_URL, params=params, timeout=TIMEOUT_GET)
    res.raise_for_status()
    body = res.text

    with open(rawf, "w", encoding="utf-8", errors="replace") as f:
        f.write(body)

    status, hitsflag = parse_status_hits(body)
    log(f"[{time.strftime('%F %T')}] SearchInfo: Status={status} ThereAreHits={hitsflag} tries={tries}")

    if status in ("FAILED", "UNKNOWN"):
        status_screen("FAILED", f"RID={RID}\nStatus={status}\nraw saved -> {rawf}")
        raise RuntimeError(f"Search {RID} failed (Status={status}).")

    if status == "READY" and hitsflag == "no":
        status_screen("NO_HITS", f"RID={RID}\nStatus=READY ThereAreHits=no\nraw saved -> {rawf}")
        raise RuntimeError(f"Search completed but found no hits (RID={RID}).")

    if status == "READY" and hitsflag == "yes":
        status_screen("READY", f"RID={RID}\nStatus=READY ThereAreHits=yes\ntries={tries}")
        break

    status_screen(phase, f"RID={RID}\ntry={tries}\nStatus={status}  ThereAreHits={hitsflag}\nraw saved -> {rawf}\nnext poll in {POLL_SEC}s")
    time.sleep(max(1, int(POLL_SEC)))

# 3) DOWNLOAD (Tabular)
phase = "DOWNLOAD (Tabular)"
log(f"[{time.strftime('%F %T')}] {phase} RID={RID}")
status_screen(phase, f"RID={RID}\nRequesting Tabular...")

get_params = {
    "CMD": "Get",
    "RID": RID,
    "FORMAT_OBJECT": "Alignment",
    "FORMAT_TYPE": "Text",
    "ALIGNMENT_VIEW": "Tabular",
    "HITLIST_SIZE": str(N),
    "ALIGNMENTS": str(N),
    "DESCRIPTIONS": str(N),
    "EMAIL": EMAIL,
    "TOOL": TOOL,
}

res = sess.get(BLAST_URL, params=get_params, timeout=TIMEOUT_GET)
res.raise_for_status()
raw = res.text

with open(rawf, "w", encoding="utf-8", errors="replace") as f:
    f.write(raw)

text = extract_pre(raw)
lines = tabular_lines(text)

if not looks_like_tabular(lines):
    status_screen("BAD_OUTPUT", f"RID={RID}\nNo valid tabular (outfmt 6) lines were detected.\nraw saved -> {rawf}")
    raise RuntimeError("Downloaded content does not look like tabular results; raw response was saved to rawf for inspection.")

with open(outf, "w", encoding="utf-8") as f:
    for ln in lines:
        f.write(ln + "\n")

elapsed = time.time() - t0
log(f"[{time.strftime('%F %T')}] DONE RID={RID} tries={tries} rows={len(lines)} bytes={os.path.getsize(outf)} elapsed={elapsed:.1f}s")
status_screen("DONE", f"RID={RID}\ntries={tries}\nrows={len(lines)}\nelapsed={elapsed:.1f}s\nwrote: {outf}\nraw saved -> {rawf}")
print("\n[FINISHED]")


In [ ]:
#@title 4) Map BLAST hits to genomes (IPG) and download ±FLANK nuccore neighborhoods (FASTA)
#@markdown Inputs: the BLASTP TSV produced in the previous cell (variable **outf**), or a file path you specify below.
#@markdown Output: per-hit neighborhood FASTA files under **wp_flanks_fna/**.


# ---- user-configurable params ----
FLANK = 100000 #@param {type:"integer"}
THREADS = 10 #@param {type:"integer"}

TOP_N_UNIQ_TAXID = 1 #@param {type:"integer"}


# ---- imports ----
import os
NCBI_EMAIL = os.environ.get('NCBI_EMAIL', globals().get('EMAIL',''))
NCBI_API_KEY = os.environ.get('NCBI_API_KEY','')

import re, time, threading, xml.etree.ElementTree as ET
from pathlib import Path
from dataclasses import dataclass
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed

from tqdm.auto import tqdm
from Bio import Entrez, SeqIO

# -------------------- AUTO / FIXED CONFIG --------------------
blast_tsv = globals().get("outf", "blastp.top5000.tsv")

outdir = Path("wp_flanks_fna")
wp_list_txt = Path("wp.unique.col2.txt")
picked_list_txt = Path("picked.unique_taxid.items.tsv")

PREFER_REFSEQ = True
DO_ORG_LOOKUP = True

NCBI_TOOL    = "symlab_wp_flanks_notebook"
ENTREZ_MAX_TRIES = 5
ENTREZ_SLEEP_BETWEEN_TRIES = 180
ENTREZ_TIMEOUT = 20
# ------------------------------------------------------------

# ---- validate ----
NCBI_EMAIL = (NCBI_EMAIL or "").strip()
NCBI_API_KEY = (NCBI_API_KEY or "").strip()
if not NCBI_EMAIL:
    raise ValueError("NCBI_EMAIL is required for NCBI Entrez access.")
if THREADS < 1:
    raise ValueError("THREADS must be >= 1")
if TOP_N_UNIQ_TAXID < 0:
    raise ValueError("TOP_N_UNIQ_TAXID must be >= 0 (0 means no limit)")

# ---- configure Entrez ----
Entrez.email = os.environ.get('NCBI_EMAIL', globals().get('EMAIL', ''))
Entrez.api_key = os.environ.get('NCBI_API_KEY', '') or None
Entrez.tool = NCBI_TOOL
Entrez.max_tries = ENTREZ_MAX_TRIES
Entrez.sleep_between_tries = ENTREZ_SLEEP_BETWEEN_TRIES
Entrez.timeout = ENTREZ_TIMEOUT

outdir.mkdir(parents=True, exist_ok=True)

# -------------------- Helpers --------------------
def normalize_wp(s: str):
    """Extract WP_XXXXX.Y from BLAST sseqid"""
    if not s:
        return None
    s = s.strip()
    m = re.search(r"\b(WP_\d+\.\d+)\b", s)
    if m:
        return m.group(1)
    for tok in s.split("|"):
        tok = tok.strip()
        if re.fullmatch(r"WP_\d+\.\d+", tok):
            return tok
    return None

def _clean_for_filename(name: str, max_len: int = 120) -> str:
    name = (name or "").strip()
    name = re.sub(r"[^\w\.\-]+", "_", name)
    name = re.sub(r"_+", "_", name).strip("_")
    if not name:
        name = "Unknown_organism"
    if len(name) > max_len:
        name = name[:max_len].rstrip("_")
    return name

def _clean_text(name: str) -> str:
    name = (name or "").strip()
    name = re.sub(r"\s+", " ", name)
    return name

class RateLimiter:
    """Global rate limit: ~3 rps without an API key; ~10 rps with an API key (more stable with threads)."""
    def __init__(self, rps: float):
        self.min_interval = 1.0 / max(0.1, rps)
        self._lock = threading.Lock()
        self._next_time = 0.0
    def wait(self):
        with self._lock:
            now = time.monotonic()
            if now < self._next_time:
                time.sleep(self._next_time - now)
            self._next_time = time.monotonic() + self.min_interval

rps = 10.0 if NCBI_API_KEY else 3.0
RL = RateLimiter(rps=rps)

MAX_RETRIES = 3
RETRY_SLEEP = 5

def entrez_efetch_limited(**kwargs):
    last_err = None
    for i in range(1, MAX_RETRIES + 1):
        try:
            RL.wait()
            return Entrez.efetch(**kwargs)
        except Exception as e:
            last_err = e
            if i == MAX_RETRIES:
                raise
            time.sleep(RETRY_SLEEP)
    raise last_err

@dataclass
class CdsMapping:
    wp: str
    accver: str
    start: int
    stop: int
    strand: str
    taxid: str = ""
    org: str = ""

_ipg_cache_lock = threading.Lock()
_ipg_cache: dict[str, list[CdsMapping]] = {}

def fetch_ipg_mappings_for_wp(wp: str) -> list[CdsMapping]:
    with _ipg_cache_lock:
        if wp in _ipg_cache:
            return _ipg_cache[wp]

    with entrez_efetch_limited(db="protein", id=wp, rettype="ipg", retmode="xml") as h:
        xml_bytes = h.read()
    root = ET.fromstring(xml_bytes)

    out: list[CdsMapping] = []
    for cds in root.iter("CDS"):
        accver = cds.attrib.get("accver", "")
        s = cds.attrib.get("start", "")
        t = cds.attrib.get("stop", "")
        if not (accver and s and t):
            continue
        try:
            start = int(s)
            stop  = int(t)
        except ValueError:
            continue
        out.append(CdsMapping(
            wp=wp,
            accver=accver,
            start=start,
            stop=stop,
            strand=cds.attrib.get("strand", "?"),
            taxid=cds.attrib.get("taxid", "") or "",
            org=cds.attrib.get("org", "") or "",
        ))

    with _ipg_cache_lock:
        _ipg_cache[wp] = out
    return out

def choose_representative_mapping(mappings: list[CdsMapping]) -> CdsMapping | None:
    if not mappings:
        return None

    cand = mappings
    if PREFER_REFSEQ:
        refseq = [m for m in cand if m.accver.startswith(("NC_", "NZ_"))]
        if refseq:
            cand = refseq

    for m in cand:
        if (m.taxid or "").strip():
            return m
    return cand[0]

def fetch_nuccore_fasta_segment(accver: str, seg_start: int, seg_end: int):
    with entrez_efetch_limited(
        db="nuccore",
        id=accver,
        rettype="fasta",
        retmode="text",
        seq_start=seg_start,
        seq_stop=seg_end,
    ) as h:
        return SeqIO.read(h, "fasta")

def _get_organism_from_gb(acc: str, seg_start: int, seg_end: int) -> str:
    try:
        with entrez_efetch_limited(
            db="nuccore",
            id=acc,
            rettype="gb",
            retmode="text",
            seq_start=seg_start,
            seq_stop=seg_end,
        ) as h:
            gb_rec = SeqIO.read(h, "gb")
        org = gb_rec.annotations.get("organism") or ""
        if not org:
            for feat in getattr(gb_rec, "features", []):
                if feat.type == "source":
                    org = feat.qualifiers.get("organism", [""])[0]
                    if org:
                        break
        return org or ""
    except Exception:
        return ""

def _fallback_org_from_fasta_desc(record) -> str:
    desc = record.description or ""
    if record.id and desc.startswith(record.id):
        desc = desc[len(record.id):].strip()
    desc = re.sub(r"bases\s+\d+\s+to\s+\d+", "", desc, flags=re.I).strip()
    m = re.match(r"(.+?)(?:,|$)", desc)
    return (m.group(1).strip() if m else "").strip()

_file_locks = defaultdict(threading.Lock)

def append_record_to_file(path: Path, fasta_text: str):
    path.parent.mkdir(parents=True, exist_ok=True)
    lock = _file_locks[str(path)]
    with lock:
        with open(path, "a", encoding="utf-8") as f:
            f.write(fasta_text)
            if not fasta_text.endswith("\n"):
                f.write("\n")

_accver_meta_lock = threading.Lock()
_accver_meta_cache: dict[str, tuple[str, str]] = {}  # accver -> (organism, taxid)

def make_outfile_path(organism: str, taxid: str, accver: str) -> Path:
    org_clean = _clean_for_filename(_clean_text(organism), max_len=80)
    taxid_clean = _clean_for_filename(taxid or "NA", max_len=32)
    acc_clean = _clean_for_filename(accver, max_len=64)
    return outdir / f"{org_clean}__taxid{taxid_clean}__{acc_clean}.fna"

def make_simple_header(organism: str, wp: str, accver: str, seg_start: int, seg_end: int, taxid: str) -> str:
    org_clean = _clean_for_filename(_clean_text(organism), max_len=120) or "Unknown_organism"
    taxid_txt = (taxid or "NA").strip()
    return f">{org_clean}|{wp}|{accver}|{seg_start}-{seg_end}|taxid={taxid_txt}"

def process_one_item(item: CdsMapping) -> tuple[int, int]:
    wp = item.wp
    accver = item.accver
    taxid = (item.taxid or "").strip()

    seg_start = max(1, min(item.start, item.stop) - int(FLANK))
    seg_end   = max(item.start, item.stop) + int(FLANK)

    try:
        record = fetch_nuccore_fasta_segment(accver, seg_start, seg_end)

        with _accver_meta_lock:
            cached = _accver_meta_cache.get(accver)

        if cached:
            organism, cached_taxid = cached
            use_taxid = taxid or cached_taxid or ""
        else:
            organism = ""
            if DO_ORG_LOOKUP:
                organism = _get_organism_from_gb(accver, seg_start, seg_end)
            if not organism:
                organism = item.org or _fallback_org_from_fasta_desc(record) or "Unknown organism"
            use_taxid = taxid or ""
            with _accver_meta_lock:
                _accver_meta_cache[accver] = (organism, use_taxid)

        outpath = make_outfile_path(organism, use_taxid, accver)
        header = make_simple_header(organism, wp, accver, seg_start, seg_end, use_taxid)
        fasta_text = f"{header}\n{record.seq}\n"
        append_record_to_file(outpath, fasta_text)
        return (1, 0)
    except Exception:
        return (0, 1)

# -------------------- Step A: read BLAST in order -> map to taxid -> dedup by taxid -> take first N unique taxids --------------------
if not Path(blast_tsv).exists():
    raise FileNotFoundError(f"BLAST TSV not found: {blast_tsv}")

print(f"[CONFIG] blast_tsv        = {blast_tsv}")
print(f"[CONFIG] FLANK(bp)        = {FLANK}  (segment = CDS ± FLANK)")
print(f"[CONFIG] THREADS          = {THREADS}")
print(f"[CONFIG] TOP_N_UNIQ_TAXID = {TOP_N_UNIQ_TAXID}  (0=all)")
print(f"[CONFIG] NCBI_EMAIL       = {NCBI_EMAIL}")
print(f"[CONFIG] API_KEY          = {'set' if NCBI_API_KEY else 'empty'}  (rps={rps})")
print(f"[CONFIG] PREFER_REFSEQ    = {PREFER_REFSEQ}")

seen_wp = set()
seen_taxid = set()
picked: list[CdsMapping] = []
picked_wp_list: list[str] = []

total_lines = 0
valid_hit_rows = 0
wp_with_no_ipg = 0
wp_ipg_error = 0

t_pick0 = time.time()

with open(blast_tsv, "r", encoding="utf-8", errors="replace") as f:
    for ln in f:
        total_lines += 1
        ln = ln.rstrip("\n")
        if not ln or ln.startswith("#"):
            continue
        cols = ln.split("\t")
        if len(cols) < 2:
            continue

        valid_hit_rows += 1
        wp = normalize_wp(cols[1])
        if not wp or wp in seen_wp:
            continue
        seen_wp.add(wp)

        try:
            mappings = fetch_ipg_mappings_for_wp(wp)
        except Exception:
            wp_ipg_error += 1
            continue

        rep = choose_representative_mapping(mappings)
        if rep is None:
            wp_with_no_ipg += 1
            continue

        taxid = (rep.taxid or "").strip()
        taxid_key = taxid if taxid else f"NA::{wp}"

        if taxid_key in seen_taxid:
            continue

        seen_taxid.add(taxid_key)
        picked.append(rep)
        picked_wp_list.append(wp)

        if TOP_N_UNIQ_TAXID and len(picked) >= TOP_N_UNIQ_TAXID:
            break

elapsed_pick = time.time() - t_pick0

wp_list_txt.write_text("\n".join(picked_wp_list) + ("\n" if picked_wp_list else ""), encoding="utf-8")

picked_list_txt.write_text(
    "wp\taccver\tstart\tstop\tstrand\ttaxid\torg\n" +
    "\n".join(
        f"{x.wp}\t{x.accver}\t{x.start}\t{x.stop}\t{x.strand}\t{(x.taxid or '').strip()}\t{(x.org or '').strip()}"
        for x in picked
    ) + ("\n" if picked else ""),
    encoding="utf-8"
)

print(f"[BLAST] total_lines={total_lines}, valid_hit_rows={valid_hit_rows}")
print(f"[PICK]  unique_taxid_picked={len(picked)} (scan stop={'YES' if (TOP_N_UNIQ_TAXID and len(picked)>=TOP_N_UNIQ_TAXID) else 'NO'})")
print(f"[PICK]  wp seen={len(seen_wp)}; ipg_error={wp_ipg_error}; no_ipg={wp_with_no_ipg}; pick_elapsed={elapsed_pick:.1f}s")
print(f"[OUT]   wp list -> {wp_list_txt}")
print(f"[OUT]   picked items -> {picked_list_txt}")

if not picked:
    raise RuntimeError("No records were selected. Possible causes: failed to parse WP IDs from the BLAST 2nd column, or IPG queries failed / returned no mapping.")

# -------------------- Step B: threaded download for picked unique-taxid representatives --------------------
t0 = time.time()
total_succ, total_fail = 0, 0

with ThreadPoolExecutor(max_workers=THREADS) as pool:
    futures = [pool.submit(process_one_item, it) for it in picked]
    for fu in tqdm(as_completed(futures), total=len(futures), desc="Downloading unique taxids", unit="item"):
        s, f_ = fu.result()
        total_succ += s
        total_fail += f_

elapsed = time.time() - t0
print(f"\nWrote {total_succ} neighborhoods; failed: {total_fail}")
print(f"Output directory：{outdir.resolve()}")
print(f"⏱️ download elapsed: {elapsed:.1f}s")
print("FASTA header format: >Organism_with_underscores|WP_xxx|ACCVER|start-end|taxid=xxx")


In [ ]:
#@title 5) antiSMASH (micromamba): run antiSMASH on downloaded neighborhoods (micromamba, fixed MPLBACKEND, STREAM OUTPUT, prereqs OK; use blast not ncbi-blast)
import os, re, hashlib, subprocess, threading, shlex
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

# Avoid matplotlib_inline backend issues inside antiSMASH
os.environ["MPLBACKEND"] = "agg"

INPUT_DIR="wp_flanks_fna"    #@param {type:"string"}
OUTROOT="antismash_out"      #@param {type:"string"}
MAX_JOBS=5                   #@param {type:"integer"}
CPUS_PER_JOB=1               #@param {type:"integer"}
DOWNLOAD_DB=True             #@param {type:"boolean"}

# Stream full antiSMASH stdout/stderr to screen (will interleave when MAX_JOBS>1)
STREAM_ANTISMASH=False       #@param {type:"boolean"}

MM = Path("/content/bin/micromamba")
ROOT = Path("/content/mm")
ENV = ROOT / "envs" / "antismash8"
DBDIR = Path("/content/antismash_db")

# Make micromamba consistent
os.environ["MAMBA_ROOT_PREFIX"] = str(ROOT)

_print_lock = threading.Lock()
def _p(msg: str):
    with _print_lock:
        print(msg, flush=True)

def run_bash(cmd: str, name="bash", stream=True):
    """Run bash; optionally stream output to notebook; raise with last 200 lines on failure."""
    cmd = (
        "export MPLBACKEND=agg\n"
        f"export MAMBA_ROOT_PREFIX={shlex.quote(str(ROOT))}\n"
        + cmd
    )

    if not stream:
        p = subprocess.run(["bash", "-lc", cmd], text=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        if p.returncode != 0:
            tail = "\n".join((p.stdout or "").splitlines()[-200:])
            raise RuntimeError(f"[{name}] failed (exit={p.returncode}). Last 200 lines:\n{tail}")
        return p.stdout

    p = subprocess.Popen(["bash", "-lc", cmd], text=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, bufsize=1)
    last = []
    assert p.stdout is not None
    for line in p.stdout:
        line = line.rstrip("\n")
        _p(f"[{name}] {line}")
        last.append(line)
        if len(last) > 200:
            last = last[-200:]
    rc = p.wait()
    if rc != 0:
        raise RuntimeError(f"[{name}] failed (exit={rc}). Last 200 lines:\n" + "\n".join(last))
    return "\n".join(last)

def safe_name(p: Path) -> str:
    s = re.sub(r"[^\w.\-]+", "_", p.stem)
    s = re.sub(r"_+", "_", s).strip("_")
    if len(s) <= 140:
        return s
    h = hashlib.md5(str(p).encode("utf-8")).hexdigest()[:10]
    return s[:120] + "_" + h

# ------------------ 1) Install micromamba ------------------
_p("[1/4] Installing micromamba (if needed)...")
run_bash(r"""
set -eo pipefail
mkdir -p /content/bin
if [ ! -x /content/bin/micromamba ]; then
  curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj -C /content/bin --strip-components=1 bin/micromamba
  chmod +x /content/bin/micromamba
fi
/content/bin/micromamba --version
""", "micromamba", stream=True)

# ------------------ 2) Create env + install antismash + prerequisites ------------------
_p("[2/4] Creating env antismash8 and installing antiSMASH 8 + prerequisites (if needed)...")
run_bash(fr"""
set -eo pipefail
mkdir -p "{ROOT}"

# (A) create base env first (easier to solve)
if [ ! -x "{ENV}/bin/antismash" ]; then
  "{MM}" create -y -p "{ENV}" -c conda-forge -c bioconda python=3.11 antismash=8
fi

# (B) then ensure external executables exist
# NOTE: use 'blast' (Bioconda) not 'ncbi-blast'
"{MM}" install -y -p "{ENV}" -c conda-forge -c bioconda \
  hmmer hmmer2 diamond blast fasttree prodigal

# Show versions + confirm tools are on PATH inside micromamba run
"{MM}" run -p "{ENV}" antismash --version

"{MM}" run -p "{ENV}" bash -lc '
set -e
for x in hmmscan hmmsearch hmmpress hmmpfam2 diamond blastp makeblastdb fasttree prodigal; do
  printf "%-12s -> " "$x"
  command -v "$x"
done
'
""", "install_antismash", stream=True)

# ------------------ 3) Optional download DB + fix default mite path ------------------
_p("[3/4] Downloading databases (optional)...")
if DOWNLOAD_DB:
    run_bash(fr"""
set -eo pipefail
mkdir -p "{DBDIR}"

# download databases to {DBDIR}
HELP="$("{MM}" run -p "{ENV}" download-antismash-databases --help 2>&1 || true)"
if echo "$HELP" | grep -q -- "--database-dir"; then
  "{MM}" run -p "{ENV}" download-antismash-databases --database-dir "{DBDIR}"
elif echo "$HELP" | grep -q -- "--install-dir"; then
  "{MM}" run -p "{ENV}" download-antismash-databases --install-dir "{DBDIR}"
else
  "{MM}" run -p "{ENV}" download-antismash-databases "{DBDIR}" || "{MM}" run -p "{ENV}" download-antismash-databases
fi

# SAFETY: some modules look in package default databases path (not your {DBDIR})
AS_DIR="$("{MM}" run -p "{ENV}" python - <<'PY'
import antismash, pathlib
print(pathlib.Path(antismash.__file__).resolve().parent)
PY
)"
PKG_DB="$AS_DIR/databases"
MITE_FASTA="$(find "{DBDIR}" -type f -name 'mite.fasta' -print -quit || true)"

if [ -n "$MITE_FASTA" ]; then
  MITE_VER_DIR="$(dirname "$MITE_FASTA")"
  MITE_ROOT="$(dirname "$MITE_VER_DIR")"
  mkdir -p "$PKG_DB"
  rm -rf "$PKG_DB/mite"
  ln -s "$MITE_ROOT" "$PKG_DB/mite"
  echo "[FIX] linked $PKG_DB/mite -> $MITE_ROOT"
else
  echo "[WARN] mite.fasta not found under {DBDIR} (genefunctions/MITE may fail)."
fi

# IMPORTANT: do a REAL prereq check (do not swallow failures)
"{MM}" run -p "{ENV}" antismash --check-prereqs --databases "{DBDIR}"
""", "download_db", stream=True)
else:
    _p("  - DOWNLOAD_DB=False, skipped.")

# ------------------ 4) Run batch ------------------
_p("[4/4] Running antiSMASH batch...")

indir = Path(INPUT_DIR)
outroot = Path(OUTROOT)
logs = Path("logs")
outroot.mkdir(parents=True, exist_ok=True)
logs.mkdir(parents=True, exist_ok=True)

files = sorted(indir.glob("*.fna"))
if not files:
    raise FileNotFoundError(f"No .fna files found under: {indir.resolve()}")

# Detect zip flag via micromamba run (ensures PATH is correct)
help_text = subprocess.check_output(
    [str(MM), "run", "-p", str(ENV), "antismash", "--help"],
    text=True,
    stderr=subprocess.STDOUT,
    env={**os.environ, "MPLBACKEND": "agg", "MAMBA_ROOT_PREFIX": str(ROOT)},
)
zip_flag = "--no-zip-output" if "--no-zip-output" in help_text else ""

BASE_ARGS = [a for a in [
    zip_flag,
    "--taxon", "bacteria",
    "--genefinding-tool", "prodigal",
    "--allow-long-headers",
] if a]

DB_ARGS = []
if DOWNLOAD_DB and DBDIR.exists():
    DB_ARGS += ["--databases", str(DBDIR)]
    DB_ARGS += [
        "--tfbs", "--cb-general", "--cb-knownclusters", "--cb-subclusters",
        "--fullhmmer", "--rre", "--cc-mibig", "--tigrfam", "--asf", "--pfam2go",
        "--smcog-trees", "--clusterhmmer",
    ]
else:
    if DOWNLOAD_DB:
        _p(f"[WARN] DOWNLOAD_DB=True but {DBDIR} not found; running without DB-dependent modules.")

COMMON_ARGS = [*BASE_ARGS, *DB_ARGS]

def run_one(f: Path):
    name = safe_name(f)
    outdir = outroot / name
    idx = outdir / "index.html"
    if idx.exists():
        _p(f"[SKIP] {f.name} (already has index.html)")
        return ("skipped", str(f))

    outdir.mkdir(parents=True, exist_ok=True)

    # IMPORTANT: run via micromamba so PATH includes ENV/bin
    cmd = [
        str(MM), "run", "-p", str(ENV),
        "antismash",
        *COMMON_ARGS,
        "-c", str(int(CPUS_PER_JOB)),
        "--output-dir", str(outdir),
        str(f),
    ]

    env = {**os.environ, "MPLBACKEND": "agg", "MAMBA_ROOT_PREFIX": str(ROOT)}

    _p(f"[RUN ] {f.name} -> {outdir}")

    if STREAM_ANTISMASH:
        p = subprocess.Popen(cmd, text=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, bufsize=1, env=env)
        last = []
        assert p.stdout is not None
        for line in p.stdout:
            line = line.rstrip("\n")
            _p(f"[{f.name}] {line}")
            last.append(line)
            if len(last) > 300:
                last = last[-300:]
        rc = p.wait()
        out = "\n".join(last)  # store tail only
        err = ""
    else:
        p = subprocess.run(cmd, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, env=env)
        rc, out, err = p.returncode, (p.stdout or ""), (p.stderr or "")

    (logs / f"{name}.stdout.log").write_text(out, encoding="utf-8")
    (logs / f"{name}.stderr.log").write_text(err, encoding="utf-8")

    if rc != 0 or not idx.exists():
        _p(f"[FAIL] {f.name} (exit={rc})")
        return ("failed", str(f))

    _p(f"[ OK ] {f.name}")
    return ("ok", str(f))

ok = failed = skipped = 0
failed_paths = []

with ThreadPoolExecutor(max_workers=int(MAX_JOBS)) as pool:
    futs = [pool.submit(run_one, f) for f in files]
    for fu in as_completed(futs):
        status, fn = fu.result()
        if status == "ok":
            ok += 1
        elif status == "failed":
            failed += 1
            failed_paths.append(fn)
        else:
            skipped += 1

failed_list = logs / "failed_files.txt"
failed_list.write_text("\n".join(failed_paths) + ("\n" if failed_paths else ""), encoding="utf-8")

_p(f"\nDone. ok={ok}, failed={failed}, skipped={skipped}")
_p(f"OUTROOT: {outroot.resolve()}")
_p(f"Logs: {logs.resolve()}")
_p(f"Failed list: {failed_list.resolve()}")


In [ ]:
#@title 6) cblaster: upload query FASTA, build local DB from antiSMASH region GBKs, and run a local search
#@markdown This cell installs cblaster (via micromamba), builds a Diamond database from antiSMASH region GenBank files, and runs cblaster search.


# ---- user-configurable params ----
U  = 3      #@param {type:"integer"}
MH = 3      #@param {type:"integer"}
MI = 20     #@param {type:"integer"}
MC = 50     #@param {type:"integer"}
HS = 10000  #@param {type:"integer"}


# ---- fixed paths/settings ----
import os
NCBI_EMAIL = os.environ.get('NCBI_EMAIL', globals().get('EMAIL','')).strip()
NCBI_API_KEY = os.environ.get('NCBI_API_KEY','').strip()

from pathlib import Path
import os, re, shlex, subprocess, threading

MAMBA_ROOT = Path("/content/mm")
MM_BIN     = Path("/content/bin/micromamba")
ENV        = MAMBA_ROOT / "envs" / "antismash8"

ANTISMASH_OUTROOT = Path("antismash_out")
REGION_PATTERNS   = ("*region*.gbk", "*region*.gbff", "*region*.gb")

DB_DIR  = Path("/content/pfaA-cblasterdb")
DB_NAME = "pfaa"

OUTPUT_DECIMALS  = 2
BINARY_DELIMITER = ","
STREAM_OUTPUT    = True

# ---- validate (same style as your WP cell) ----
NCBI_EMAIL = (NCBI_EMAIL or "").strip()
NCBI_API_KEY = (NCBI_API_KEY or "").strip()

if not NCBI_EMAIL:
    raise ValueError("NCBI_EMAIL is required for cblaster config / Entrez settings.")

if U < 1 or MH < 1 or MI < 0 or MC < 0 or HS < 0:
    raise ValueError("Invalid parameter ranges: u/mh>=1; mi/mc/hs>=0.")

# ---- helpers ----
_print_lock = threading.Lock()
def _p(msg: str):
    with _print_lock:
        print(msg, flush=True)

def run_bash(cmd: str, name="bash", stream=True, check=True):
    cmd = f"export MAMBA_ROOT_PREFIX={shlex.quote(str(MAMBA_ROOT))}\n" + cmd
    if not stream:
        p = subprocess.run(["bash", "-lc", cmd], text=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        out = p.stdout or ""
        if check and p.returncode != 0:
            tail = "\n".join(out.splitlines()[-200:])
            raise RuntimeError(f"[{name}] failed (exit={p.returncode}). Last 200 lines:\n{tail}")
        return out

    p = subprocess.Popen(["bash", "-lc", cmd], text=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, bufsize=1)
    last = []
    assert p.stdout is not None
    for line in p.stdout:
        line = line.rstrip("\n")
        _p(f"[{name}] {line}")
        last.append(line)
        if len(last) > 200:
            last = last[-200:]
    rc = p.wait()
    if check and rc != 0:
        raise RuntimeError(f"[{name}] failed (exit={rc}). Last 200 lines:\n" + "\n".join(last))
    return "\n".join(last)

def mm_run(args, name="mm", stream=True, check=True):
    if not MM_BIN.exists():
        raise FileNotFoundError(f"micromamba not found: {MM_BIN}. Run the antiSMASH/micromamba setup cell first.")
    if not ENV.exists():
        raise FileNotFoundError(f"Environment not found: {ENV}. Run the antiSMASH environment creation cell first.")

    cmd = [str(MM_BIN), "run", "-p", str(ENV), *args]
    env = {**os.environ, "MAMBA_ROOT_PREFIX": str(MAMBA_ROOT)}

    if not stream:
        p = subprocess.run(cmd, text=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, env=env)
        out = p.stdout or ""
        if check and p.returncode != 0:
            tail = "\n".join(out.splitlines()[-200:])
            raise RuntimeError(f"[{name}] failed (exit={p.returncode}). Last 200 lines:\n{tail}")
        return out

    p = subprocess.Popen(cmd, text=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, bufsize=1, env=env)
    last = []
    assert p.stdout is not None
    for line in p.stdout:
        line = line.rstrip("\n")
        _p(f"[{name}] {line}")
        last.append(line)
        if len(last) > 300:
            last = last[-300:]
    rc = p.wait()
    if check and rc != 0:
        raise RuntimeError(f"[{name}] failed (exit={rc}). Last 300 lines:\n" + "\n".join(last))
    return "\n".join(last)

def sanitize_prefix(s: str, fallback="query"):
    s = re.sub(r"[^\w.\-]+", "_", (s or "").strip())
    s = re.sub(r"_+", "_", s).strip("_")
    return s[:80] if s else fallback

def ensure_query_fasta(upload_path: Path, out_fa: Path) -> str:
    txt = upload_path.read_text(encoding="utf-8", errors="ignore").strip()
    if not txt:
        raise ValueError(f"Uploaded file is empty: {upload_path}")

    if txt.lstrip().startswith(">"):
        lines = [ln.rstrip() for ln in txt.splitlines() if ln.strip() != ""]
        header = lines[0][1:].strip()
        prefix = sanitize_prefix(header.split()[0] if header else out_fa.stem)
        out_fa.write_text("\n".join(lines) + "\n", encoding="utf-8")
        return prefix

    seq = re.sub(r"\s+", "", txt)
    if re.search(r"[^A-Za-z\*\-\.]", seq):
        bad = sorted(set(re.findall(r"[^A-Za-z\*\-\.]", seq)))
        raise ValueError(f"Uploaded sequence contains unsupported characters: {bad}")
    prefix = sanitize_prefix(upload_path.stem, "query")
    out_fa.write_text(f">{prefix}\n{seq}\n", encoding="utf-8")
    return prefix

def collect_region_files(root: Path):
    region_files = []
    for pat in REGION_PATTERNS:
        region_files.extend([p for p in root.rglob(pat) if p.is_file()])
    return sorted(set(region_files))

def ensure_cblaster_installed():
    try:
        mm_run(["cblaster", "--version"], name="cblaster_version", stream=False, check=True)
        return
    except Exception:
        pass

    run_bash(fr"""
set -euo pipefail
"{MM_BIN}" install -y -p "{ENV}" -c conda-forge -c bioconda cblaster diamond blast
"{MM_BIN}" run -p "{ENV}" cblaster --version
"{MM_BIN}" run -p "{ENV}" diamond version || true
"{MM_BIN}" run -p "{ENV}" bash -lc 'blastp -version | head -n 1' || true
""", name="install_cblaster", stream=STREAM_OUTPUT, check=True)

def ensure_cblaster_config(email: str, api_key: str):
    help_txt = mm_run(["cblaster", "config", "-h"], name="cblaster_config_help", stream=False, check=False)
    email_flag = "--email" if "--email" in help_txt else None
    apikey_flag = "--api_key" if "--api_key" in help_txt else ("--api-key" if "--api-key" in help_txt else None)
    force_flag = "--force" if "--force" in help_txt else ("-f" if re.search(r"(?m)^\s*-f[, ]", help_txt) else None)

    if not email_flag:
        raise RuntimeError("Could not detect the --email option from `cblaster config -h` output.")

    cmd = ["cblaster", "config"]
    if force_flag:
        cmd += [force_flag]
    cmd += [email_flag, email]
    if api_key and apikey_flag:
        cmd += [apikey_flag, api_key]

    mm_run(cmd, name="cblaster_config", stream=STREAM_OUTPUT, check=True)

def build_db_if_needed(region_files):
    DB_DIR.mkdir(parents=True, exist_ok=True)
    dmnd = DB_DIR / f"{DB_NAME}.dmnd"
    sqlite3 = DB_DIR / f"{DB_NAME}.sqlite3"
    fasta = DB_DIR / f"{DB_NAME}.fasta"

    if dmnd.exists() and sqlite3.exists() and fasta.exists():
        _p(f"[INFO] DB exists; reuse: {dmnd}")
        return dmnd, sqlite3, fasta

    _p(f"[INFO] region files found: {len(region_files)}")
    _p(f"[INFO] building DB: {DB_DIR}/{DB_NAME}.*")

    stage = DB_DIR / "_stage_gbk"
    stage.mkdir(parents=True, exist_ok=True)
    for x in stage.glob("*"):
        try:
            x.unlink()
        except Exception:
            pass

    for i, p in enumerate(region_files, 1):
        suf = p.suffix if p.suffix else ".gbk"
        (stage / f"r{i:06d}{suf}").symlink_to(p)

    cmd = f'cd {shlex.quote(str(DB_DIR))} && cblaster makedb -n {shlex.quote(DB_NAME)} _stage_gbk/*'
    mm_run(["bash", "-lc", cmd], name="cblaster_makedb", stream=STREAM_OUTPUT, check=True)

    if not (dmnd.exists() and sqlite3.exists() and fasta.exists()):
        raise RuntimeError(f"DB build finished but outputs missing in {DB_DIR}: {dmnd.name}, {sqlite3.name}, {fasta.name}")

    return dmnd, sqlite3, fasta

def run_local_search(query_fa: Path, dmnd: Path, prefix: str):
    plot_html    = Path(f"my_plot-{prefix}.html").resolve()
    session_json = Path(f"session-{prefix}.json").resolve()
    summary_csv  = Path(f"summary-{prefix}.csv").resolve()
    abspres_csv  = Path(f"abspres-{prefix}.csv").resolve()

    args = [
        "cblaster", "search",
        "-qf", str(query_fa),
        "--plot", str(plot_html),
        "--session_file", str(session_json),
        "--output", str(summary_csv),
        "--output_decimals", str(int(OUTPUT_DECIMALS)),
        "--binary", str(abspres_csv),
        "--binary_delimiter", str(BINARY_DELIMITER),
        "-u", str(int(U)),
        "-mh", str(int(MH)),
        "-mi", str(int(MI)),
        "-mc", str(int(MC)),
        "-hs", str(int(HS)),
        "-db", str(dmnd),
        "--mode", "local",
    ]

    _p("[RUN] " + " ".join(shlex.quote(x) for x in args))
    mm_run(args, name="cblaster_search", stream=STREAM_OUTPUT, check=True)

    _p("\n Done.")
    _p(f"📄 Plot:    {plot_html}")
    _p(f" Session: {session_json}")
    _p(f"📑 Summary: {summary_csv}")
    _p(f"📊 Binary:  {abspres_csv}")

# -------------------- MAIN --------------------
print(f"[CONFIG] NCBI_EMAIL = {NCBI_EMAIL}")
print(f"[CONFIG] API_KEY    = {'set' if NCBI_API_KEY else 'empty'}")
print(f"[CONFIG] -u {U}  -mh {MH}  -mi {MI}  -mc {MC}  -hs {HS}")

# Step 1: upload query
_p("[1/4] Upload query FASTA (or AA .txt)...")
try:
    from google.colab import files
    uploaded = files.upload()
except Exception as e:
    raise RuntimeError(f"Colab upload not available: {e}")
if not uploaded:
    raise RuntimeError("No file uploaded.")

up_name = next(iter(uploaded.keys()))
up_path = Path("/content") / up_name
query_fa = Path("/content/cblaster_query.faa")
prefix = ensure_query_fasta(up_path, query_fa)
prefix = sanitize_prefix(prefix, "query")
_p(f"[INFO] query = {query_fa}  prefix={prefix}")

# Step 2: ensure tool
_p("[2/4] Ensure cblaster installed...")
ensure_cblaster_installed()

# Step 3: ensure config (this fixes your previous error)
_p("[3/4] Run cblaster config (required by cblaster 1.4.x)...")
ensure_cblaster_config(NCBI_EMAIL, NCBI_API_KEY)

# Step 4: build DB + search
_p("[4/4] Build/validate local DB from antiSMASH region GBKs, then search...")
root = ANTISMASH_OUTROOT.resolve()
if not root.exists():
    raise FileNotFoundError(f"antiSMASH output root not found: {root}")

region_files = collect_region_files(root)
if not region_files:
    raise FileNotFoundError(f"No region GenBanks found under {root} matching {REGION_PATTERNS}")

dmnd, sqlite3, fasta = build_db_if_needed(region_files)
run_local_search(query_fa, dmnd, prefix)


In [ ]:
#@title 7) Rule-based filter and copy passing antiSMASH region folders/files
#@markdown This cell reads a cblaster abs/presence CSV/TSV (e.g., **abspres-*.csv**) to select antiSMASH region GenBank files and applies a four-step rule-based filter.
#@markdown It then copies either the matching GBK files or their parent directories into **DEST**.


import os, re, shutil, time
from pathlib import Path

# ---------------- user-configurable params ----------------
INPUT = ""
ROOT  = ""
DEST  = "final-target"
COPY_MODE = "dir"
COL = 1
PROTO_GAP_MAX = 25000  #@param {type:"integer"}
OVERWRITE = False
DRY_RUN = False

# ---------------- helpers ----------------
try:
    from tqdm.auto import tqdm
except Exception:
    def tqdm(x, **k):  # fallback
        return x

def _p(msg: str):
    print(msg, flush=True)

def detect_fs(first_line: str):
    if "\t" in first_line:
        return "\t"
    if "," in first_line:
        return ","
    return None  # whitespace-split

def trim_cell(s: str) -> str:
    s = str(s)
    s = s.lstrip("\ufeff")  # BOM
    s = s.strip().strip('"').strip("'").strip()
    s = re.sub(r"\r$", "", s)
    return s

def normalize_region_base(s: str):
    s = trim_cell(s)
    if not s:
        return None
    s = s.split("#", 1)[0].strip()
    s = re.sub(r"^.*\/", "", s)  # drop path
    s = re.sub(r"\.(gbk|gbff|gb)$", "", s, flags=re.I)
    m = re.search(r"([^\s,;\"']+\.region\d+)$", s)
    if m:
        return m.group(1)
    return None

def extract_region_names(input_path: Path, col: int = 1):
    txt = input_path.read_text(encoding="utf-8", errors="ignore").splitlines()
    first = next((ln for ln in txt if ln.strip()), "")
    if not first:
        raise ValueError(f"Input file is empty: {input_path}")
    fs = detect_fs(first)

    names = set()

    # ---- by column (preferred) ----
    if fs is not None:
        for i, ln in enumerate(txt, start=1):
            if not ln.strip():
                continue
            parts = ln.split(fs)
            if len(parts) < col:
                continue
            v = trim_cell(parts[col-1])
            # skip header-like first line
            if i == 1 and v.lower() in {"organism","scaffold","name","filename"}:
                continue
            b = normalize_region_base(v)
            if b:
                names.add(b)
    else:
        # whitespace
        for i, ln in enumerate(txt, start=1):
            if not ln.strip():
                continue
            parts = re.split(r"\s+", ln.strip())
            if len(parts) < col:
                continue
            v = trim_cell(parts[col-1])
            if i == 1 and v.lower() in {"organism","scaffold","name","filename"}:
                continue
            b = normalize_region_base(v)
            if b:
                names.add(b)

    # ---- fallback regex if empty ----
    if not names:
        blob = "\n".join(txt)
        for m in re.finditer(r"([^\s,;\"']+\.region\d+)", blob):
            b = normalize_region_base(m.group(1))
            if b:
                names.add(b)

    return sorted(names)

def index_region_files(root: Path):
    exts = {".gbk",".gbff",".gb"}
    base2paths = {}
    for dirpath, _, filenames in os.walk(root):
        for fn in filenames:
            suf = Path(fn).suffix.lower()
            if suf not in exts:
                continue
            p = Path(dirpath) / fn
            base = re.sub(r"\.(gbk|gbff|gb)$", "", fn, flags=re.I)
            base2paths.setdefault(base, []).append(p)
    return base2paths

# ---- step3 parsing: protocluster core_location + PFAM_domain de-dup ----
_core_re = re.compile(r'/core_location="?\[(\d+):(\d+)\]\([+-]\)"?')
_prod_re = re.compile(r'/product="([^"]+)"')

_pfam_re   = re.compile(r'/db_xref="(PF\d+)\.\d+"')
_asdom_re  = re.compile(r'/aSDomain="([^"]+)"')
_ps_re     = re.compile(r'/protein_start="(\d+)"')
_pe_re     = re.compile(r'/protein_end="(\d+)"')

def _is_qual(line: str) -> bool:
    return line.startswith("                     ")  # 21 spaces

def parse_protoclusters_core(lines):
    clusters = []
    i, n = 0, len(lines)
    while i < n:
        if lines[i].startswith("     protocluster"):
            cur = {"product": None, "core_start": None, "core_end": None}
            i += 1
            while i < n and _is_qual(lines[i]):
                s = lines[i].strip()
                m = _prod_re.search(s)
                if m:
                    cur["product"] = m.group(1)
                m = _core_re.search(s)
                if m:
                    a, b = int(m.group(1)), int(m.group(2))
                    cur["core_start"], cur["core_end"] = (a, b) if a <= b else (b, a)
                i += 1
            if cur["product"] and cur["core_start"] is not None and cur["core_end"] is not None:
                clusters.append(cur)
            continue
        i += 1
    return clusters

def gap_half_open(a_start, a_end, b_start, b_end) -> int:
    # treat [start:end] as half-open [start, end)
    left = max(a_start, b_start)
    right = min(a_end, b_end)
    if left < right:
        return 0
    if a_end <= b_start:
        return b_start - a_end
    if b_end <= a_start:
        return a_start - b_end
    return 0

def parse_unique_pfam_domains(lines):
    """
    Only PFAM_domain features are parsed; deduplicate by (location, PF set, aSDomain, protein_start, protein_end),
    to remove duplicates between fullhmmer and clusterhmmer.
    """
    domains = set()
    i, n = 0, len(lines)
    while i < n:
        if lines[i].startswith("     PFAM_domain"):
            loc = lines[i][len("     PFAM_domain"):].strip()
            pfams = set()
            asdom = ""
            ps = None
            pe = None
            i += 1
            while i < n and _is_qual(lines[i]):
                s = lines[i].strip()
                m = _pfam_re.search(s)
                if m:
                    pfams.add(m.group(1))
                m = _asdom_re.search(s)
                if m:
                    asdom = m.group(1) or ""
                m = _ps_re.search(s)
                if m:
                    ps = int(m.group(1))
                m = _pe_re.search(s)
                if m:
                    pe = int(m.group(1))
                i += 1
            key = (loc, tuple(sorted(pfams)), asdom, ps, pe)
            domains.add(key)
            continue
        i += 1
    return domains

def count_domains(domains):
    pf00501_n = 0
    amp_n = 0
    cond_n = 0

    for loc, pfams, asdom, ps, pe in domains:
        pfams = set(pfams)
        asdom_l = (asdom or "").lower()

        if "PF00501" in pfams:
            pf00501_n += 1
        if asdom_l == "amp-binding":
            amp_n += 1

        # Condensation: count ONCE per PFAM_domain feature
        is_cond = ("PF00668" in pfams) or (asdom_l in {"condensation_lcl","condensation_dcl"})
        if is_cond:
            cond_n += 1

    return pf00501_n, amp_n, cond_n

def passes_filters(gbk_path: Path) -> bool:
    text = gbk_path.read_text(encoding="utf-8", errors="ignore")

    # step1
    if re.search(r"PKS_AT", text, flags=re.I) is None:
        return False
    # step2
    if re.search(r"ketoacyl synthase", text, flags=re.I) is None:
        return False

    lines = text.splitlines()

    # step3b domain counts (PFAM_domain only)
    domains = parse_unique_pfam_domains(lines)
    pf00501_n, amp_n, cond_n = count_domains(domains)
    if not ((pf00501_n >= 2 or amp_n >= 2) and cond_n >= 2):
        return False

    # step3a protocluster core_location gap
    clusters = parse_protoclusters_core(lines)
    nrps = [c for c in clusters if (c["product"] or "").strip().upper() == "NRPS"]
    hgl  = [c for c in clusters if "HGLE-KS" in (c["product"] or "").strip().upper()]
    if not nrps or not hgl:
        return False

    min_gap = None
    for a in nrps:
        for b in hgl:
            g = gap_half_open(a["core_start"], a["core_end"], b["core_start"], b["core_end"])
            min_gap = g if min_gap is None else min(min_gap, g)
    if min_gap is None or min_gap > int(PROTO_GAP_MAX):
        return False

    # step4 exact /product="NRPS"
    if '/product="NRPS"' not in text:
        return False

    return True

def write_list(p: Path, items):
    p.parent.mkdir(parents=True, exist_ok=True)
    p.write_text("\n".join(map(str, items)) + ("\n" if items else ""), encoding="utf-8")

# ---------------- auto-resolve paths ----------------
prefix = globals().get("prefix", "query")
ANTISMASH_OUTROOT = globals().get("ANTISMASH_OUTROOT", Path("antismash_out"))

root = Path(ROOT.strip()) if str(ROOT).strip() else Path(ANTISMASH_OUTROOT)
root = root.resolve()
dest = Path(DEST).resolve()

if not INPUT.strip():
    cand = [Path(f"abspres-{prefix}.csv"), Path(f"summary-{prefix}.csv")]
    cand = [c for c in cand if c.exists()]
    if not cand:
        raise FileNotFoundError("INPUT is empty and no abspres-*.csv / summary-*.csv was found. Please provide INPUT as a file path.")
    input_path = cand[0].resolve()
else:
    input_path = Path(INPUT).expanduser().resolve()

_p(f"[INFO] INPUT={input_path}")
_p(f"[INFO] ROOT ={root}")
_p(f"[INFO] DEST ={dest}")
_p(f"[INFO] COPY_MODE={COPY_MODE}  COL={COL}  PROTO_GAP_MAX={PROTO_GAP_MAX}")
_p(f"[INFO] OVERWRITE={OVERWRITE}  DRY_RUN={DRY_RUN}")

if not input_path.exists():
    raise FileNotFoundError(f"INPUT not found: {input_path}")
if not root.exists():
    raise FileNotFoundError(f"ROOT not found: {root}")

dest.mkdir(parents=True, exist_ok=True)

t0 = time.time()

# 1) names
names = extract_region_names(input_path, col=int(COL))
write_list(dest / "extracted_names.txt", names)
_p(f"[INFO] extracted names: {len(names)}  -> {dest/'extracted_names.txt'}")
_p(f"[INFO] first 3 names: {names[:3]}")

# 2) index + match
_p("[INFO] indexing region files under ROOT ...")
base2paths = index_region_files(root)
all_bases = sorted(base2paths.keys())
write_list(dest / "all_region_bases.txt", all_bases)

matched = []
for nm in names:
    matched.extend(base2paths.get(nm, []))
matched = sorted(set(matched))
write_list(dest / "matched_region_files.abs.txt", [str(p) for p in matched])

unmatched = sorted(set(names) - set(base2paths.keys()))
write_list(dest / "unmatched_names.txt", unmatched)

_p(f"[INFO] matched gbk files: {len(matched)} -> {dest/'matched_region_files.abs.txt'}")
_p(f"[INFO] unmatched names:   {len(unmatched)} -> {dest/'unmatched_names.txt'}")

if not matched:
    _p("[WARN] No matched gbk files. Stop.")
    raise SystemExit(0)

# 3) filters
kept = []
_p("[INFO] filtering matched gbk files ...")
for p in tqdm(matched, total=len(matched)):
    try:
        if passes_filters(p):
            kept.append(p)
    except Exception:
        continue

kept = sorted(set(kept))
write_list(dest / "kept_region_files.abs.txt", [str(p) for p in kept])

_p(f"[INFO] kept gbk files: {len(kept)} -> {dest/'kept_region_files.abs.txt'}")

if not kept:
    _p("[INFO] No files passed filters; nothing to copy.")
    raise SystemExit(0)

# 4) decide targets
if COPY_MODE == "dir":
    targets = sorted({p.parent for p in kept})
else:
    targets = kept[:]

write_list(dest / "kept_targets.abs.txt", [str(p) for p in targets])

# 5) copy
if DRY_RUN:
    _p("[DRY_RUN] skip copying.")
else:
    _p("[INFO] copying targets ...")
    for src in tqdm(targets, total=len(targets)):
        src = Path(src)
        try:
            rel = src.relative_to(root)
        except Exception:
            rel = src.name

        dst = dest / rel
        if dst.exists():
            if not OVERWRITE:
                continue
            if dst.is_dir():
                shutil.rmtree(dst)
            else:
                dst.unlink()

        dst.parent.mkdir(parents=True, exist_ok=True)
        if src.is_dir():
            shutil.copytree(src, dst, dirs_exist_ok=True)
        else:
            shutil.copy2(src, dst)

dt = time.time() - t0
_p(f"[DONE] elapsed: {dt:.1f}s")
_p(f"Extracted names:  {len(names)}  -> {dest/'extracted_names.txt'}")
_p(f"Matched gbk:      {len(matched)} -> {dest/'matched_region_files.abs.txt'}")
_p(f"Kept gbk:         {len(kept)} -> {dest/'kept_region_files.abs.txt'}")
_p(f"Copied targets:   {len(targets)} -> {dest/'kept_targets.abs.txt'}")
_p(f"Unmatched names:  {len(unmatched)} -> {dest/'unmatched_names.txt'}")


In [ ]:
#@title 8) Diagnostics (optional): explain why no GBKs passed the filter
#@markdown Use this cell only if the previous filtering step yields zero hits. It summarizes reasons and writes a TSV report.


import re, os
from pathlib import Path

DEST = "/content/final-target"  #@param {type:"string"}
PROTO_GAP_MAX = 25000           #@param {type:"integer"}
SHOW_TOP = 15

dest = Path(DEST)
matched_list = dest / "matched_region_files.abs.txt"
out_tsv = dest / "filter_diagnosis.tsv"

if not matched_list.exists():
    raise FileNotFoundError(f"Not found: {matched_list}")

files = [Path(x.strip()) for x in matched_list.read_text().splitlines() if x.strip()]
print(f"[INFO] diagnosing {len(files)} files ...")

# ---- reuse parsing logic (same as filter cell) ----
_core_re = re.compile(r'/core_location="?\[(\d+):(\d+)\]\([+-]\)"?')
_prod_re = re.compile(r'/product="([^"]+)"')
_pfam_re   = re.compile(r'/db_xref="(PF\d+)\.\d+"')
_asdom_re  = re.compile(r'/aSDomain="([^"]+)"')
_ps_re     = re.compile(r'/protein_start="(\d+)"')
_pe_re     = re.compile(r'/protein_end="(\d+)"')

def _is_qual(line: str) -> bool:
    return line.startswith("                     ")  # 21 spaces

def parse_protoclusters_core(lines):
    clusters = []
    i, n = 0, len(lines)
    while i < n:
        if lines[i].startswith("     protocluster"):
            cur = {"product": None, "core_start": None, "core_end": None}
            i += 1
            while i < n and _is_qual(lines[i]):
                s = lines[i].strip()
                m = _prod_re.search(s)
                if m:
                    cur["product"] = m.group(1)
                m = _core_re.search(s)
                if m:
                    a, b = int(m.group(1)), int(m.group(2))
                    cur["core_start"], cur["core_end"] = (a, b) if a <= b else (b, a)
                i += 1
            if cur["product"] and cur["core_start"] is not None and cur["core_end"] is not None:
                clusters.append(cur)
            continue
        i += 1
    return clusters

def gap_half_open(a_start, a_end, b_start, b_end) -> int:
    left = max(a_start, b_start)
    right = min(a_end, b_end)
    if left < right:
        return 0
    if a_end <= b_start:
        return b_start - a_end
    if b_end <= a_start:
        return a_start - b_end
    return 0

def parse_unique_pfam_domains(lines):
    domains = set()
    i, n = 0, len(lines)
    while i < n:
        if lines[i].startswith("     PFAM_domain"):
            loc = lines[i][len("     PFAM_domain"):].strip()
            pfams = set()
            asdom = ""
            ps = None
            pe = None
            i += 1
            while i < n and _is_qual(lines[i]):
                s = lines[i].strip()
                m = _pfam_re.search(s)
                if m:
                    pfams.add(m.group(1))
                m = _asdom_re.search(s)
                if m:
                    asdom = m.group(1) or ""
                m = _ps_re.search(s)
                if m:
                    ps = int(m.group(1))
                m = _pe_re.search(s)
                if m:
                    pe = int(m.group(1))
                i += 1
            key = (loc, tuple(sorted(pfams)), asdom, ps, pe)
            domains.add(key)
            continue
        i += 1
    return domains

def count_domains(domains):
    pf00501_n = 0
    amp_n = 0
    cond_n = 0

    for loc, pfams, asdom, ps, pe in domains:
        pfams = set(pfams)
        asdom_l = (asdom or "").lower()

        if "PF00501" in pfams:
            pf00501_n += 1
        if asdom_l == "amp-binding":
            amp_n += 1

        is_cond = ("PF00668" in pfams) or (asdom_l in {"condensation_lcl","condensation_dcl"})
        if is_cond:
            cond_n += 1

    return pf00501_n, amp_n, cond_n

def diagnose_one(p: Path):
    try:
        text = p.read_text(errors="ignore")
    except Exception as e:
        return dict(file=str(p), readable=0, err=str(e))

    step1 = 1 if re.search(r"PKS_AT", text, flags=re.I) else 0
    step2 = 1 if re.search(r"ketoacyl synthase", text, flags=re.I) else 0
    step4 = 1 if '/product="NRPS"' in text else 0

    lines = text.splitlines()
    domains = parse_unique_pfam_domains(lines)
    pf00501_n, amp_n, cond_n = count_domains(domains)
    step3_domains = 1 if ((pf00501_n >= 2 or amp_n >= 2) and cond_n >= 2) else 0

    clusters = parse_protoclusters_core(lines)
    nrps = [c for c in clusters if (c["product"] or "").strip().upper() == "NRPS"]
    hgl  = [c for c in clusters if "HGLE-KS" in (c["product"] or "").strip().upper()]
    has_nrps = 1 if nrps else 0
    has_hgle = 1 if hgl else 0

    min_gap = ""
    step3_gap = 0
    if nrps and hgl:
        mg = None
        for a in nrps:
            for b in hgl:
                g = gap_half_open(a["core_start"], a["core_end"], b["core_start"], b["core_end"])
                mg = g if mg is None else min(mg, g)
        min_gap = mg
        step3_gap = 1 if (mg is not None and mg <= int(PROTO_GAP_MAX)) else 0

    passed = 1 if (step1 and step2 and step3_domains and step3_gap and step4) else 0

    reasons = []
    if not step1: reasons.append("fail_step1(PKS_AT)")
    if not step2: reasons.append("fail_step2(ketoacyl_synthase)")
    if not step3_domains: reasons.append(f"fail_step3_domains(PF00501={pf00501_n},AMP={amp_n},COND={cond_n})")
    if not has_nrps: reasons.append("fail_step3_gap(no_NRPS_protocluster)")
    if not has_hgle: reasons.append("fail_step3_gap(no_hglE-KS_protocluster)")
    if has_nrps and has_hgle and not step3_gap: reasons.append(f"fail_step3_gap(min_gap={min_gap} > {PROTO_GAP_MAX})")
    if not step4: reasons.append('fail_step4(no_exact_/product="NRPS")')
    reason_str = ";".join(reasons) if reasons else ""

    return dict(
        file=str(p),
        readable=1,
        step1=step1, step2=step2, step3_domains=step3_domains, step3_gap=step3_gap, step4=step4, passed=passed,
        pf00501=pf00501_n, amp=amp_n, cond=cond_n,
        protocluster_nrps=len(nrps), protocluster_hgle=len(hgl),
        min_gap=min_gap,
        reason=reason_str
    )

rows = [diagnose_one(p) for p in files]

cols = [
    "file","readable","passed",
    "step1","step2","step3_domains","step3_gap","step4",
    "pf00501","amp","cond",
    "protocluster_nrps","protocluster_hgle","min_gap",
    "reason"
]
out_lines = ["\t".join(cols)]
for r in rows:
    out_lines.append("\t".join(str(r.get(c,"")) for c in cols))
out_tsv.write_text("\n".join(out_lines) + "\n", encoding="utf-8")
print(f"[INFO] wrote: {out_tsv}")

total = len(rows)
passed_n = sum(1 for r in rows if r.get("passed")==1)
print(f"[SUMMARY] total={total}  passed={passed_n}")

def count_fail(key):
    return sum(1 for r in rows if r.get(key)==0 and r.get("readable")==1)

print("[SUMMARY] fail counts (among readable files):")
print("  step1(PKS_AT):", count_fail("step1"))
print("  step2(ketoacyl synthase):", count_fail("step2"))
print("  step3_domains:", count_fail("step3_domains"))
print("  step3_gap:", count_fail("step3_gap"))
print("  step4(/product=\"NRPS\"):", count_fail("step4"))

fails = [r for r in rows if r.get("readable")==1 and r.get("passed")==0]
print(f"\n[EXAMPLES] showing up to {SHOW_TOP} fails:")
for r in fails[:SHOW_TOP]:
    print("-", Path(r["file"]).name)
    print("  ", r["reason"])


In [ ]:
#@title 9) Clinker: visualize only the kept GBKs (deduplicated)
#@markdown Runs clinker on the GBKs that passed filtering and generates an interactive HTML and a session JSON.


import os, subprocess, shutil, threading, http.server, socketserver, socket
from pathlib import Path

# ====== config ======
DEST = Path(globals().get("DEST", "final-target")).resolve()
KEEP_LIST = DEST / "kept_region_files.abs.txt"

OUTDIR = DEST / "clinker_kept_out"
INPUTS = OUTDIR / "inputs"
PLOT_HTML = OUTDIR / "clinker_kept.html"
SESSION_JSON = OUTDIR / "clinker_kept.session.json"

JOBS = "0"
FORCE = True
MAX_CLUSTERS = 0
# ====================

import sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-U", "clinker"])

if not KEEP_LIST.exists():
    raise FileNotFoundError(f"Not found: {KEEP_LIST}")

# 1) read kept list + dedup
raw = [x.strip() for x in KEEP_LIST.read_text().splitlines() if x.strip()]
gbk_paths = [Path(x) for x in raw]
gbk_paths = sorted({p.resolve() for p in gbk_paths if p.exists()})

print(f"[INFO] kept gbk files (raw lines) = {len(raw)}")
print(f"[INFO] kept gbk files (exists+dedup) = {len(gbk_paths)}")

if not gbk_paths:
    print("[INFO] kept list empty after dedup/existence check.")
    raise SystemExit(0)

if MAX_CLUSTERS and len(gbk_paths) > MAX_CLUSTERS:
    gbk_paths = gbk_paths[:MAX_CLUSTERS]
    print(f"[INFO] capped to first {MAX_CLUSTERS} gbks")

# 2) rebuild inputs directory cleanly
OUTDIR.mkdir(parents=True, exist_ok=True)
if INPUTS.exists():
    shutil.rmtree(INPUTS)
INPUTS.mkdir(parents=True, exist_ok=True)

# 3) isolate inputs: symlink (preferred) or copy
kept_inputs = []
src_seen = {}  # src_resolved -> dst list
for i, src in enumerate(gbk_paths, start=1):
    dst = INPUTS / f"{i:05d}.{src.name}"
    try:
        os.symlink(src, dst)
    except Exception:
        shutil.copy2(src, dst)
    kept_inputs.append(dst)
    src_seen.setdefault(str(src), []).append(str(dst))

# 3.1 sanity: any source linked twice?
dups = {s: ds for s, ds in src_seen.items() if len(ds) > 1}
if dups:
    print("[WARN] same source file appears multiple times in inputs (should not happen):")
    for s, ds in dups.items():
        print("  SRC:", s)
        for d in ds:
            print("    ->", d)
else:
    print("[INFO] no duplicate sources in inputs.")

# 4) explicit list inputs/*.gbk (no shell glob)
input_files = sorted(INPUTS.glob("*.gbk"))
print(f"[INFO] input gbk files prepared = {len(input_files)}")
# 4.1 final dedup by realpath just in case
real2file = {}
for p in input_files:
    rp = str(p.resolve())
    real2file.setdefault(rp, []).append(str(p))
dup_real = {rp: ps for rp, ps in real2file.items() if len(ps) > 1}
if dup_real:
    print("[WARN] duplicate realpaths among inputs (multiple links to same file):")
    for rp, ps in dup_real.items():
        print("  REAL:", rp)
        for p in ps:
            print("    ->", p)

# 5) run clinker with explicit argv list (no shell)
cmd = ["clinker", *map(str, input_files), "-p", str(PLOT_HTML), "-s", str(SESSION_JSON), "-j", str(JOBS)]
if FORCE:
    cmd += ["-f"]

print("[INFO] running clinker with N inputs =", len(input_files))
print("[INFO] cmd head:", " ".join(cmd[:8]), "...")
subprocess.run(cmd, check=True)

print(f"[INFO] plot saved: {PLOT_HTML}")
print(f"[INFO] session saved: {SESSION_JSON}")

# 6) display once with a free port (avoid 'address already in use')
from google.colab import output
from IPython.display import display

def pick_free_port():
    s = socket.socket()
    s.bind(("", 0))
    port = s.getsockname()[1]
    s.close()
    return port

PORT = pick_free_port()

class QuietHandler(http.server.SimpleHTTPRequestHandler):
    def log_message(self, format, *args):
        pass

def serve(port):
    os.chdir(str(OUTDIR))
    with socketserver.TCPServer(("", port), QuietHandler) as httpd:
        httpd.serve_forever()

t = threading.Thread(target=serve, args=(PORT,), daemon=True)
t.start()

display(output.serve_kernel_port_as_iframe(PORT, path=f"/{PLOT_HTML.name}"))



---

### References
- NCBI BLAST URLAPI documentation (for programmatic BLAST queries).
- antiSMASH for BGC prediction.
- cblaster for local gene cluster searches.

If you distribute results, please cite the upstream tools you used (antiSMASH, cblaster, etc.) according to their licenses and publication guidance.
